In [1]:
from json import loads
from copy import deepcopy
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 80)
pd.set_option('display.max_rows', 80)

In [2]:
df = pd.read_csv('./FilteredCsv/dataset_after_level_1.csv',index_col='Id')
df

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,Ex,Y,SBrkr,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1999.0,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,Stone,119.0,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163,589,1542,GasA,TA,Y,SBrkr,2073,0,0,2073,1,0,2,0,3,1,TA,7,Min1,2,TA,Attchd,1978.0,Unf,2,500,TA,TA,Y,349,0,0,0,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,None,0.0,Ex,Gd,Stone,TA,Gd,No,GLQ,275,Unf,0,877,1152,GasA,Ex,Y,SBrkr,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,1941.0,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500


In [3]:
col_continuous = ['GrLivArea', 'LotArea', 'BsmtFinSF1', 'BsmtUnfSF', '1stFlrSF', 'TotalBsmtSF', 'GarageArea','2ndFlrSF', 'BsmtFinSF2', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'LotFrontage', 'ScreenPorch']
col_catagorial = ['Neighborhood', 'Exterior2nd', 'MSSubClass', 'Exterior1st', 'YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'MoSold', 'TotRmsAbvGrd', 'OverallQual', 'OverallCond', 'Condition1', 'SaleType', 'RoofMatl', 'BedroomAbvGr', 'Condition2', 'HouseStyle', 'Functional', 'GarageType', 'SaleCondition', 'RoofStyle', 'Electrical', 'Heating', 'BsmtFinType2', 'BsmtFinType1', 'Foundation', 'BldgType', 'MSZoning', 'GarageCond', 'YrSold', 'GarageQual', 'ExterCond', 'GarageCars', 'LotConfig', 'HeatingQC', 'Fireplaces', 'LandContour', 'KitchenQual', 'MasVnrType', 'FullBath', 'BsmtFullBath', 'KitchenAbvGr', 'LotShape', 'BsmtExposure', 'BsmtCond', 'BsmtQual', 'ExterQual', 'HalfBath','GarageFinish', 'BsmtHalfBath', 'LandSlope', 'PavedDrive', 'Utilities','Street', 'CentralAir','FireplaceQu']


In [4]:
with open('./data_dict_equations.json','rt') as f:
    data_dict_continuous = loads(f.read())
data_dict_continuous

{'2ndFlrSF': '+0.06*(x**2)+98.59*(x**1)+71631.97*(x**0)',
 'GrLivArea': '+103.69*(x**1)+22362.6*(x**0)',
 'LotArea': '+8.65*(x**1)+96145.5*(x**0)',
 'BsmtFinSF1': '+0.09*(x**2)+-26.62*(x**1)+146988.42*(x**0)',
 'BsmtUnfSF': '+18.26*(x**1)+166525.59*(x**0)',
 '1stFlrSF': '+121.53*(x**1)+37790.65*(x**0)',
 'TotalBsmtSF': '+122.09*(x**1)+48499.91*(x**0)',
 'GarageArea': '+251.97*(x**1)+56662.96*(x**0)',
 'WoodDeckSF': '+136.18*(x**1)+174518.22*(x**0)',
 'OpenPorchSF': '+130.05*(x**1)+194337.81*(x**0)',
 'BsmtFinSF2': '+29.7*(x**1)+149161.31*(x**0)',
 'EnclosedPorch': '+172.75*(x**1)+118584.92*(x**0)',
 'LotFrontage': '+1312.97*(x**1)+86274.97*(x**0)',
 'ScreenPorch': '+214.49*(x**1)+152419.64*(x**0)'}

In [5]:
df_continuous = deepcopy(df[col_continuous])
df_continuous

,GrLivArea,LotArea,BsmtFinSF1,BsmtUnfSF,1stFlrSF,TotalBsmtSF,GarageArea,2ndFlrSF,BsmtFinSF2,WoodDeckSF,OpenPorchSF,EnclosedPorch,LotFrontage,ScreenPorch
Id,,,,,,,,,,,,,,
1,1710,8450,706,150,856,856,548,854,0,0,61,0,65.0,0
2,1262,9600,978,284,1262,1262,460,0,0,298,0,0,80.0,0
3,1786,11250,486,434,920,920,608,866,0,0,42,0,68.0,0
4,1717,9550,216,540,961,756,642,756,0,0,35,272,60.0,0
5,2198,14260,655,490,1145,1145,836,1053,0,192,84,0,84.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,1647,7917,0,953,953,953,460,694,0,0,40,0,62.0,0
1457,2073,13175,790,589,2073,1542,500,0,163,349,0,0,85.0,0
1458,2340,9042,275,877,1188,1152,252,1152,0,0,60,0,66.0,0


In [6]:
df_continuous.isna().sum()

GrLivArea          0
LotArea            0
BsmtFinSF1         0
BsmtUnfSF          0
1stFlrSF           0
TotalBsmtSF        0
GarageArea         0
2ndFlrSF           0
BsmtFinSF2         0
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
LotFrontage      239
ScreenPorch        0
dtype: int64

In [7]:
df_continuous['LotFrontage'].fillna(df_continuous['LotFrontage'].mean(),inplace=True)

In [8]:
def get_saleprice_continuous(col_as_series):
        x = col_as_series.values
        return eval(data_dict_continuous[col_as_series.name])

In [9]:
df_train_continuous =df_continuous.apply(axis='index',func=get_saleprice_continuous)
df_train_continuous

,GrLivArea,LotArea,BsmtFinSF1,BsmtUnfSF,1stFlrSF,TotalBsmtSF,GarageArea,2ndFlrSF,BsmtFinSF2,WoodDeckSF,OpenPorchSF,EnclosedPorch,LotFrontage,ScreenPorch
Id,,,,,,,,,,,,,,
1,199672.50,169238.00,173053.94,169264.59,141820.33,153008.95,194742.52,199586.79,149161.31,174518.22,202270.86,118584.92,171618.02,152419.64
2,153219.38,179185.50,207037.62,171711.43,191161.51,202577.49,172569.16,71631.97,149161.31,215099.86,194337.81,118584.92,191312.57,152419.64
3,207552.94,193458.00,155308.74,174450.43,149598.25,160822.71,209860.72,202008.27,149161.31,174518.22,199799.91,118584.92,175556.93,152419.64
4,200398.33,178753.00,145437.54,176385.99,154580.98,140799.95,218427.70,180458.17,149161.31,174518.22,198889.56,165572.92,165053.17,152419.64
5,250273.22,219494.50,168164.57,175472.99,176942.50,188292.96,267309.88,241975.78,149161.31,200664.78,205262.01,118584.92,196564.45,152419.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,193140.03,164627.55,146988.42,183927.37,153608.74,164851.68,172569.16,168951.59,149161.31,174518.22,199539.81,118584.92,167679.11,152419.64
1457,237311.97,210109.25,182127.62,177280.73,289722.34,236762.69,182647.96,71631.97,154002.41,222045.04,194337.81,118584.92,197877.42,152419.64
1458,264997.20,174358.80,146474.17,182539.61,182168.29,189147.59,120159.40,264833.89,149161.31,174518.22,202140.81,118584.92,172930.99,152419.64


In [10]:
df_categorical = deepcopy(df[col_catagorial])
df_categorical

,Neighborhood,Exterior2nd,MSSubClass,Exterior1st,YearBuilt,YearRemodAdd,GarageYrBlt,MoSold,TotRmsAbvGrd,OverallQual,OverallCond,Condition1,SaleType,RoofMatl,BedroomAbvGr,Condition2,HouseStyle,Functional,GarageType,SaleCondition,RoofStyle,Electrical,Heating,BsmtFinType2,BsmtFinType1,Foundation,BldgType,MSZoning,GarageCond,YrSold,GarageQual,ExterCond,GarageCars,LotConfig,HeatingQC,Fireplaces,LandContour,KitchenQual,MasVnrType,FullBath,BsmtFullBath,KitchenAbvGr,LotShape,BsmtExposure,BsmtCond,BsmtQual,ExterQual,HalfBath,GarageFinish,BsmtHalfBath,LandSlope,PavedDrive,Utilities,Street,CentralAir,FireplaceQu
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,CollgCr,VinylSd,60,VinylSd,2003,2003,2003.0,2,8,7,5,Norm,WD,CompShg,3,Norm,2Story,Typ,Attchd,Normal,Gable,SBrkr,GasA,Unf,GLQ,PConc,1Fam,RL,TA,2008,TA,TA,2,Inside,Ex,0,Lvl,Gd,BrkFace,2,1,1,Reg,No,TA,Gd,Gd,1,RFn,0,Gtl,Y,AllPub,Pave,Y,NaN
2,Veenker,MetalSd,20,MetalSd,1976,1976,1976.0,5,6,6,8,Feedr,WD,CompShg,3,Norm,1Story,Typ,Attchd,Normal,Gable,SBrkr,GasA,Unf,ALQ,CBlock,1Fam,RL,TA,2007,TA,TA,2,FR2,Ex,1,Lvl,TA,None,2,0,1,Reg,Gd,TA,Gd,TA,0,RFn,1,Gtl,Y,AllPub,Pave,Y,TA
3,CollgCr,VinylSd,60,VinylSd,2001,2002,2001.0,9,6,7,5,Norm,WD,CompShg,3,Norm,2Story,Typ,Attchd,Normal,Gable,SBrkr,GasA,Unf,GLQ,PConc,1Fam,RL,TA,2008,TA,TA,2,Inside,Ex,1,Lvl,Gd,BrkFace,2,1,1,IR1,Mn,TA,Gd,Gd,1,RFn,0,Gtl,Y,AllPub,Pave,Y,TA
4,Crawfor,Wd Shng,70,Wd Sdng,1915,1970,1998.0,2,7,7,5,Norm,WD,CompShg,3,Norm,2Story,Typ,Detchd,Abnorml,Gable,SBrkr,GasA,Unf,ALQ,BrkTil,1Fam,RL,TA,2006,TA,TA,3,Corner,Gd,1,Lvl,Gd,None,1,1,1,IR1,No,Gd,TA,TA,0,Unf,0,Gtl,Y,AllPub,Pave,Y,Gd
5,NoRidge,VinylSd,60,VinylSd,2000,2000,2000.0,12,9,8,5,Norm,WD,CompShg,4,Norm,2Story,Typ,Attchd,Normal,Gable,SBrkr,GasA,Unf,GLQ,PConc,1Fam,RL,TA,2008,TA,TA,3,FR2,Ex,1,Lvl,Gd,BrkFace,2,1,1,IR1,Av,TA,Gd,Gd,1,RFn,0,Gtl,Y,AllPub,Pave,Y,TA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,Gilbert,VinylSd,60,VinylSd,1999,2000,1999.0,8,7,6,5,Norm,WD,CompShg,3,Norm,2Story,Typ,Attchd,Normal,Gable,SBrkr,GasA,Unf,Unf,PConc,1Fam,RL,TA,2007,TA,TA,2,Inside,Ex,1,Lvl,TA,None,2,0,1,Reg,No,TA,Gd,TA,1,RFn,0,Gtl,Y,AllPub,Pave,Y,TA
1457,NWAmes,Plywood,20,Plywood,1978,1988,1978.0,2,7,6,6,Norm,WD,CompShg,3,Norm,1Story,Min1,Attchd,Normal,Gable,SBrkr,GasA,Rec,ALQ,CBlock,1Fam,RL,TA,2010,TA,TA,2,Inside,TA,2,Lvl,TA,Stone,2,1,1,Reg,No,TA,Gd,TA,0,Unf,0,Gtl,Y,AllPub,Pave,Y,TA
1458,Crawfor,CmentBd,70,CemntBd,1941,2006,1941.0,5,9,7,9,Norm,WD,CompShg,4,Norm,2Story,Typ,Attchd,Normal,Gable,SBrkr,GasA,Unf,GLQ,Stone,1Fam,RL,TA,2010,TA,Gd,1,Inside,Ex,2,Lvl,Gd,None,2,0,1,Reg,No,Gd,TA,Ex,0,RFn,0,Gtl,Y,AllPub,Pave,Y,Gd


In [11]:
with open('./data_dict_median.json','rt') as f:
    data_dict_median = loads(f.read())

In [12]:
def get_saleprice_categorical(col_as_series):
    
    result = []
    col_as_series_str = col_as_series.astype('str')
    
    for cat_val in col_as_series_str.values:
        if cat_val in data_dict_median[col_as_series_str.name]:
            result.append(data_dict_median[col_as_series_str.name][cat_val])
        else:
            # print(f"appending {data_dict_median[col_as_series_str.name]['Other']}",col_as_series_str.name , 'for val', cat_val)
            result.append(data_dict_median[col_as_series_str.name]['Other'])
    return np.array(result)

In [13]:
df_train_categorical =df_categorical.apply(axis='index',func=get_saleprice_categorical)
df_train_categorical


,Neighborhood,Exterior2nd,MSSubClass,Exterior1st,YearBuilt,YearRemodAdd,GarageYrBlt,MoSold,TotRmsAbvGrd,OverallQual,OverallCond,Condition1,SaleType,RoofMatl,BedroomAbvGr,Condition2,HouseStyle,Functional,GarageType,SaleCondition,RoofStyle,Electrical,Heating,BsmtFinType2,BsmtFinType1,Foundation,BldgType,MSZoning,GarageCond,YrSold,GarageQual,ExterCond,GarageCars,LotConfig,HeatingQC,Fireplaces,LandContour,KitchenQual,MasVnrType,FullBath,BsmtFullBath,KitchenAbvGr,LotShape,BsmtExposure,BsmtCond,BsmtQual,ExterQual,HalfBath,GarageFinish,BsmtHalfBath,LandSlope,PavedDrive,Utilities,Street,CentralAir,FireplaceQu
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,197200.0,200070.5,215200.0,200000.0,207500.0,184900.0,204250.0,173250.0,200000.0,200141.0,185000.0,166500.0,158000.0,162000.0,169945.0,163500.0,190000.0,165500.0,185000.0,160000.0,160000.0,170000.0,164500.0,167000.0,213750.0,205000.0,167900.0,174000.0,170000.0,164000.0,170000.0,167370.0,177750.0,159697.5,194700.0,135000.0,162900.0,201400.0,181000.0,196750.0,178800.0,165550.0,146000.0,154000.0,165000.0,192070.0,220000.0,189000.0,190000.0,163250.0,161875.0,168500.0,163000.0,163000.0,168000.0,199640.0
2,218000.0,138750.0,159250.0,139000.0,150900.0,151450.0,152000.0,154000.0,155000.0,160000.0,142500.0,140000.0,158000.0,162000.0,169945.0,163500.0,154750.0,165500.0,185000.0,160000.0,160000.0,170000.0,164500.0,167000.0,149250.0,141500.0,167900.0,174000.0,170000.0,167000.0,170000.0,167370.0,177750.0,165000.0,194700.0,189975.0,162900.0,137000.0,143000.0,196750.0,154150.0,165550.0,146000.0,226975.0,165000.0,192070.0,139450.0,143000.0,190000.0,159000.0,161875.0,168500.0,163000.0,163000.0,168000.0,187500.0
3,197200.0,200070.5,215200.0,200000.0,226500.0,215750.0,226500.0,185900.0,155000.0,200141.0,185000.0,166500.0,158000.0,162000.0,169945.0,163500.0,190000.0,165500.0,185000.0,160000.0,160000.0,170000.0,164500.0,167000.0,213750.0,205000.0,167900.0,174000.0,170000.0,164000.0,170000.0,167370.0,177750.0,159697.5,194700.0,189975.0,162900.0,201400.0,181000.0,196750.0,178800.0,165550.0,189000.0,182450.0,165000.0,192070.0,220000.0,189000.0,190000.0,163250.0,161875.0,168500.0,163000.0,163000.0,168000.0,187500.0
4,200624.0,138225.0,156000.0,138943.5,139500.0,129250.0,199900.0,173250.0,184100.0,200141.0,185000.0,166500.0,158000.0,162000.0,169945.0,163500.0,190000.0,165500.0,129500.0,130000.0,160000.0,170000.0,164500.0,167000.0,149250.0,125250.0,167900.0,174000.0,170000.0,163995.0,170000.0,167370.0,295000.0,160000.0,152000.0,189975.0,162900.0,201400.0,143000.0,132375.0,178800.0,165550.0,189000.0,154000.0,193879.0,135500.0,139450.0,143000.0,135000.0,163250.0,161875.0,168500.0,163000.0,163000.0,168000.0,206950.0
5,301500.0,200070.5,215200.0,200000.0,199750.0,173000.0,181000.0,178000.0,252000.0,269750.0,185000.0,166500.0,158000.0,162000.0,193500.0,163500.0,190000.0,165500.0,185000.0,160000.0,160000.0,170000.0,164500.0,167000.0,213750.0,205000.0,167900.0,174000.0,170000.0,164000.0,170000.0,167370.0,295000.0,165000.0,194700.0,189975.0,162900.0,201400.0,181000.0,196750.0,178800.0,165550.0,189000.0,185850.0,165000.0,192070.0,220000.0,189000.0,190000.0,163250.0,161875.0,168500.0,163000.0,163000.0,168000.0,187500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,181000.0,200070.5,215200.0,200000.0,191000.0,173000.0,182450.0,174450.0,184100.0,160000.0,185000.0,166500.0,158000.0,162000.0,169945.0,163500.0,190000.0,165500.0,185000.0,160000.0,160000.0,170000.0,164500.0,167000.0,161750.0,205000.0,167900.0,174000.0,170000.0,167000.0,170000.0,167370.0,177750.0,159697.5,194700.0,189975.0,162900.0,137000.0,143000.0,196750.0,154150.0,165550.0,146000.0,154000.0,165000.0,192070.0,139450.0,189000.0,190000.0,163250.0,161875.0,168500.0,163000.0,163000.0,168000.0,187500.0
1457,182900.0,160750.0,159250.0,167450.0,152750.0,196500.0,151000

In [14]:
df_combined_continuous_categorical = df_train_continuous.merge(df_train_categorical,left_index=True,right_index=True)
df_combined_continuous_categorical

,GrLivArea,LotArea,BsmtFinSF1,BsmtUnfSF,1stFlrSF,TotalBsmtSF,GarageArea,2ndFlrSF,BsmtFinSF2,WoodDeckSF,OpenPorchSF,EnclosedPorch,LotFrontage,ScreenPorch,Neighborhood,Exterior2nd,MSSubClass,Exterior1st,YearBuilt,YearRemodAdd,GarageYrBlt,MoSold,TotRmsAbvGrd,OverallQual,OverallCond,Condition1,SaleType,RoofMatl,BedroomAbvGr,Condition2,HouseStyle,Functional,GarageType,SaleCondition,RoofStyle,Electrical,Heating,BsmtFinType2,BsmtFinType1,Foundation,BldgType,MSZoning,GarageCond,YrSold,GarageQual,ExterCond,GarageCars,LotConfig,HeatingQC,Fireplaces,LandContour,KitchenQual,MasVnrType,FullBath,BsmtFullBath,KitchenAbvGr,LotShape,BsmtExposure,BsmtCond,BsmtQual,ExterQual,HalfBath,GarageFinish,BsmtHalfBath,LandSlope,PavedDrive,Utilities,Street,CentralAir,FireplaceQu
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,199672.50,169238.00,173053.94,169264.59,141820.33,153008.95,194742.52,199586.79,149161.31,174518.22,202270.86,118584.92,171618.02,152419.64,197200.0,200070.5,215200.0,200000.0,207500.0,184900.0,204250.0,173250.0,200000.0,200141.0,185000.0,166500.0,158000.0,162000.0,169945.0,163500.0,190000.0,165500.0,185000.0,160000.0,160000.0,170000.0,164500.0,167000.0,213750.0,205000.0,167900.0,174000.0,170000.0,164000.0,170000.0,167370.0,177750.0,159697.5,194700.0,135000.0,162900.0,201400.0,181000.0,196750.0,178800.0,165550.0,146000.0,154000.0,165000.0,192070.0,220000.0,189000.0,190000.0,163250.0,161875.0,168500.0,163000.0,163000.0,168000.0,199640.0
2,153219.38,179185.50,207037.62,171711.43,191161.51,202577.49,172569.16,71631.97,149161.31,215099.86,194337.81,118584.92,191312.57,152419.64,218000.0,138750.0,159250.0,139000.0,150900.0,151450.0,152000.0,154000.0,155000.0,160000.0,142500.0,140000.0,158000.0,162000.0,169945.0,163500.0,154750.0,165500.0,185000.0,160000.0,160000.0,170000.0,164500.0,167000.0,149250.0,141500.0,167900.0,174000.0,170000.0,167000.0,170000.0,167370.0,177750.0,165000.0,194700.0,189975.0,162900.0,137000.0,143000.0,196750.0,154150.0,165550.0,146000.0,226975.0,165000.0,192070.0,139450.0,143000.0,190000.0,159000.0,161875.0,168500.0,163000.0,163000.0,168000.0,187500.0
3,207552.94,193458.00,155308.74,174450.43,149598.25,160822.71,209860.72,202008.27,149161.31,174518.22,199799.91,118584.92,175556.93,152419.64,197200.0,200070.5,215200.0,200000.0,226500.0,215750.0,226500.0,185900.0,155000.0,200141.0,185000.0,166500.0,158000.0,162000.0,169945.0,163500.0,190000.0,165500.0,185000.0,160000.0,160000.0,170000.0,164500.0,167000.0,213750.0,205000.0,167900.0,174000.0,170000.0,164000.0,170000.0,167370.0,177750.0,159697.5,194700.0,189975.0,162900.0,201400.0,181000.0,196750.0,178800.0,165550.0,189000.0,182450.0,165000.0,192070.0,220000.0,189000.0,190000.0,163250.0,161875.0,168500.0,163000.0,163000.0,168000.0,187500.0
4,200398.33,178753.00,145437.54,176385.99,154580.98,140799.95,218427.70,180458.17,149161.31,174518.22,198889.56,165572.92,165053.17,152419.64,200624.0,138225.0,156000.0,138943.5,139500.0,129250.0,199900.0,173250.0,184100.0,200141.0,185000.0,166500.0,158000.0,162000.0,169945.0,163500.0,190000.0,165500.0,129500.0,130000.0,160000.0,170000.0,164500.0,167000.0,149250.0,125250.0,167900.0,174000.0,170000.0,163995.0,170000.0,167370.0,295000.0,160000.0,152000.0,189975.0,162900.0,201400.0,143000.0,132375.0,178800.0,165550.0,189000.0,154000.0,193879.0,135500.0,139450.0,143000.0,135000.0,163250.0,161875.0,168500.0,163000.0,163000.0,168000.0,206950.0
5,250273.22,219494.50,168164.57,175472.99,176942.50,188292.96,267309.88,241975.78,149161.31,200664.78,205262.01,118584.92,196564.45,152419.64,301500.0,200070.5,215200.0,200000.0,199750.0,173000.0,181000.0,178000.0,252000.0,269750.0,185000.0,166500.0,158000.0,162000.0,193500.0,163500.0,190000.0,165500.0,185000.0,160000.0,160000.0,170000.0,164500.0,167000.0,213750.0,205000.0,167900.0,174000.0,170000.0,164000.0,170000.0,167370.0,295000.0,165000.0,194700.0,189975.0,162900.0,201400.0,181000.0,196750.0,178800.0,165550.0,189000.0,185850.0,165000.0,192070.0,220000.0,189000.0,190000.0,

In [16]:
df_combined_continuous_categorical.isna().sum()

GrLivArea        0
LotArea          0
BsmtFinSF1       0
BsmtUnfSF        0
1stFlrSF         0
TotalBsmtSF      0
GarageArea       0
2ndFlrSF         0
BsmtFinSF2       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
LotFrontage      0
ScreenPorch      0
Neighborhood     0
Exterior2nd      0
MSSubClass       0
Exterior1st      0
YearBuilt        0
YearRemodAdd     0
GarageYrBlt      0
MoSold           0
TotRmsAbvGrd     0
OverallQual      0
OverallCond      0
Condition1       0
SaleType         0
RoofMatl         0
BedroomAbvGr     0
Condition2       0
HouseStyle       0
Functional       0
GarageType       0
SaleCondition    0
RoofStyle        0
Electrical       0
Heating          0
BsmtFinType2     0
BsmtFinType1     0
Foundation       0
BldgType         0
MSZoning         0
GarageCond       0
YrSold           0
GarageQual       0
ExterCond        0
GarageCars       0
LotConfig        0
HeatingQC        0
Fireplaces       0
LandContour      0
KitchenQual      0
MasVnrType  

In [17]:
df_combined_continuous_categorical.apply(axis='index',func=lambda x:(x==0).sum())

GrLivArea        0
LotArea          0
BsmtFinSF1       0
BsmtUnfSF        0
1stFlrSF         0
TotalBsmtSF      0
GarageArea       0
2ndFlrSF         0
BsmtFinSF2       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
LotFrontage      0
ScreenPorch      0
Neighborhood     0
Exterior2nd      0
MSSubClass       0
Exterior1st      0
YearBuilt        0
YearRemodAdd     0
GarageYrBlt      0
MoSold           0
TotRmsAbvGrd     0
OverallQual      0
OverallCond      0
Condition1       0
SaleType         0
RoofMatl         0
BedroomAbvGr     0
Condition2       0
HouseStyle       0
Functional       0
GarageType       0
SaleCondition    0
RoofStyle        0
Electrical       0
Heating          0
BsmtFinType2     0
BsmtFinType1     0
Foundation       0
BldgType         0
MSZoning         0
GarageCond       0
YrSold           0
GarageQual       0
ExterCond        0
GarageCars       0
LotConfig        0
HeatingQC        0
Fireplaces       0
LandContour      0
KitchenQual      0
MasVnrType  

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

x_train, x_test, y_train, y_test =train_test_split(df_combined_continuous_categorical.to_numpy(),df['SalePrice'].values.reshape(-1,1),test_size=0.2)

In [19]:
lr = LinearRegression()
lr.fit(x_train,y_train)
lr.score(x_test,y_test)

0.9109243667150146

In [20]:
lr.coef_

array([[-0.02788903,  0.1143687 ,  0.08769267, -0.71052199,  0.42148867,
         0.16844007, -0.00335999,  0.29646282, -0.22559387,  0.05246033,
         0.17899028,  0.06979988,  0.08073137,  0.13959282,  0.15119078,
        -0.03069847, -0.06036169,  0.0537394 ,  0.05060374,  0.08597505,
        -0.01301211, -0.08620326,  0.03724624,  0.2750316 , -0.05079526,
         0.19909024,  0.03341248, -0.13602312, -0.00801175, -0.15417789,
         0.00506658,  0.46890863, -0.03540602,  0.17873569,  0.0380842 ,
        -0.05806975,  0.0356474 ,  0.14782595,  0.02769205, -0.03767662,
         0.10502456,  0.04760719,  0.07187203,  0.25737704,  0.04042935,
        -0.03541271,  0.09806243,  0.10865081,  0.02058987,  0.07531455,
         0.12974908,  0.06969991, -0.00221997,  0.006188  ,  0.12077058,
         0.56638138, -0.01131802,  0.11527367,  0.1326061 ,  0.05892465,
         0.07095863,  0.08218433, -0.00927913, -0.16606177,  0.0890178 ,
        -0.02627507,  1.65546535,  0.38108944,  0.0

In [21]:
rfr = RandomForestRegressor()
rfr.fit(x_train,y_train.flatten())
rfr.score(x_test,y_test)

0.8829406513773893

In [26]:
df_feature_weight = pd.DataFrame({
                                'Features':df_combined_continuous_categorical.columns,
                                'Weights' :lr.coef_.flatten()
                                    }).sort_values(by='Weights',ascending=False)
df_feature_weight

,Features,Weights
66,Utilities,1.655465
55,KitchenAbvGr,0.566381
31,Functional,0.468909
4,1stFlrSF,0.421489
67,Street,0.381089
7,2ndFlrSF,0.296463
23,OverallQual,0.275032
43,YrSold,0.257377
25,Condition1,0.199090
10,OpenPorchSF,0.178990


In [27]:
df_feature_weight['Weights']

5.909209075731507

In [28]:
lr.coef_.flatten().sum()

5.909209075731507

In [29]:
lr.intercept_

array([-794159.47746154])

In [30]:
lr = LinearRegression(fit_intercept=False)
lr.fit(x_train,y_train)
lr.score(x_test,y_test)

0.9097194687331944

In [31]:
lr.coef_

array([[-0.00719515,  0.11452742,  0.03572112, -0.9752004 ,  0.41081283,
         0.21214112, -0.00234236,  0.28703529, -0.51651035,  0.0481923 ,
         0.13921013,  0.06477625,  0.0772206 ,  0.11451469,  0.14831813,
        -0.03897878, -0.05709888,  0.05791728,  0.05657999,  0.08500812,
        -0.01773278, -0.08616778,  0.03279565,  0.28351274, -0.048432  ,
         0.19711851,  0.03502436, -0.15603928, -0.02029629, -0.18676175,
        -0.00580917,  0.44554585, -0.03597545,  0.18226405,  0.00497233,
        -0.04648292,  0.00952675,  0.11526732,  0.00667978, -0.02696532,
         0.12128625,  0.04087775,  0.05574917,  0.12359678,  0.03300616,
        -0.03885678,  0.09915656,  0.06185825,  0.02400611,  0.07187398,
         0.1245259 ,  0.06514073, -0.01046099,  0.01054522,  0.12628085,
         0.51926235, -0.01094711,  0.12154808,  0.11586546,  0.06269422,
         0.07499657,  0.09530079, -0.01092487, -1.40585208,  0.01111315,
        -0.03906722, -0.56755395,  0.13719302,  0.0

In [32]:
lr.coef_.flatten().sum()

1.0415595630032906

In [33]:
df_feature_weight = pd.DataFrame({
                                'Features':df_combined_continuous_categorical.columns,
                                'Weights' :lr.coef_.flatten()
                                    }).sort_values(by='Weights',ascending=False)
df_feature_weight

,Features,Weights
55,KitchenAbvGr,0.519262
31,Functional,0.445546
4,1stFlrSF,0.410813
7,2ndFlrSF,0.287035
23,OverallQual,0.283513
5,TotalBsmtSF,0.212141
25,Condition1,0.197119
33,SaleCondition,0.182264
14,Neighborhood,0.148318
10,OpenPorchSF,0.139210
